# Sudoku solver

In [76]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [77]:
def cross(a, b):
    return [s + t for s in a for t in b]

## Setup boxes, units and peers

Let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:

- The individual squares at the intersection of rows and columns will be called __boxes__. These boxes will have labels 'A1', 'A2', ..., 'I9'.
- The complete rows, columns, and 3x3 squares, will be called __units__. Thus, each unit is a set of 9 boxes, and there are 27 units in total.
- For a particular box (such as 'A1'), its __peers__ will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square).

In [78]:
boxes = cross(rows, cols)

In [79]:
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') 
                              for cs in ('123','456','789')]

unitlist = row_units + column_units + square_units

In [80]:
#units_dict = dict((b, [u for u in unit_list if b in u]) for b in boxes)

In [81]:
def create_units_dict():

    units_dict = dict(zip(boxes, [[] for _ in boxes]))
    
    for unit in unitlist:
        for box in unit:
            units_dict[box].append(unit)

    return units_dict

units_dict = create_units_dict()

In [82]:
#peers_dict = dict((s, set(sum(units_dict[s], [])) - set([s])) for s in boxes)

In [83]:
def create_peers_dict():

    peers_dict = {}
    
    for box in boxes:
        blist = [b for u in units_dict[box] for b in u]
        box_peers = set(blist) - set([box])
        peers_dict[box] = box_peers
        
    return peers_dict

peers_dict = create_peers_dict()

In [84]:
def display(values):

    width = 1 + max(len(values[s]) for s in boxes)

    line = '+'.join(['-' * (width * 3)] * 3)

    for r in rows:
        print(''.join(values[r + c].center(width) 
                      + ('|' if c in '36' else '') for c in cols))
        if r in 'CF': 
            print(line)

In [85]:
def grid_values(grid):
    
    assert len(grid) == len(boxes)
    
    return dict(zip(boxes, [v for v in grid]))

In [86]:
def init_state(values):
    
    values_string = '123456789'

    for box in boxes:
        if values[box] == ".":
            values[box] = values_string

In [87]:
def eliminate(values):

    solved_boxes = [box for box in values.keys() if len(values[box]) == 1]

    for box in solved_boxes:
        for peer in peers_dict[box]:
            values[peer] = values[peer].replace(values[box], '')

In [88]:
def only_choice(values):
    
    values_string = '123456789'

    for unit in unitlist:

        value_blist_dict = {v:[] for v in values_string}

        for box in unit:
            for v in values[box]:
                value_blist_dict[v].append(box)
        
        for v in value_blist_dict:
            blist = value_blist_dict[v]
            if len(blist) == 1:
                values[blist[0]] = v

## Psuedocode for checking for naked twins...

**function** NakedTwins(_values_) **returns** a dict mapping from Sudoku box names to a list of feasible values  
&emsp;**inputs:**  
&emsp;&emsp;_values_, a dict mapping from Sudoku box names to a list of feasible values  
&emsp;_out_ <- **copy**(_values_)  /* make a deep copy */  
&emsp;**for each** _boxA_ in _values_ **do**  
&emsp;&emsp;**for each** _boxB_ of **PEERS**(_boxA_) **do**  
&emsp;&emsp;&emsp;**if** both _values_[_boxA_] and _values_[_boxB_] exactly match and have only two feasible digits **do**  
&emsp;&emsp;&emsp;&emsp;**for each** _peer_ of **INTERSECTION**(**PEERS**(_boxA_), **PEERS**(_boxB_)) **do**  
&emsp;&emsp;&emsp;&emsp;&emsp;**for each** _digit_ of _values_[_boxA_] **do**  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;remove digit _d_ from _out_[_peer_]  
&emsp;**return** _out_

In [90]:
def remove(vstr1, vstr2):
    for v in vstr1:
        vstr2 = vstr2.replace(v, '')
    return vstr2

In [98]:
def naked_twins(values):
    
    for b1 in boxes:
        values_b1 = values[b1]
        if len(values_b1) == 2:
            for b2 in peers_dict[b1]:
                if values[b2] == values_b1:
                    for b12_peer in (peers_dict[b1] & peers_dict[b2]):
                        values[b12_peer] = remove(values_b1, values[b12_peer])

In [92]:
def reduce_puzzle(values):

    stalled = False
    while not stalled:

        solved_count_before = len([box for box in values.keys() 
                                   if len(values[box]) == 1])

        eliminate(values)
        only_choice(values)
        naked_twins(values)

        solved_count_after = len([box for box in values.keys() 
                                  if len(values[box]) == 1])

        stalled = (solved_count_before == solved_count_after)

        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False

    return True

In [93]:
def search_puzzle(values):
    
    if not reduce_puzzle(values):
        return False

    if all(len(values[b]) == 1 for b in boxes): 
        return True
    
    _, b = min((len(values[b]), b) for b in boxes if len(values[b]) > 1)
    
    for v in values[b]:

        test_values = values.copy()
        test_values[b] = v

        if search_puzzle(test_values):
            values.update(test_values)
            return True
    
    return False

In [94]:
def solve_puzzle(grid):

    values = grid_values(grid)

    print("Initial puzzle state...")
    display(values)
    print()

    init_state(values)
    if search_puzzle(values):
        print("Final puzzle state...")
        display(values)
    else:
        print("Couldn't solve puzzle.")

In [99]:
puzzle1 = '..3.2.6..9..3.5..1..18.64..'\
          '..81.29..7.......8..67.82..'\
          '..26.95..8..2.3..9..5.1.3..'

solve_puzzle(puzzle1)

Initial puzzle state...
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 

Found twins B3 and H3 with values 47
Found twins B3 and B5 with values 47
Peer B8 reduced from 278 to 28
Found twins B5 and B3 with values 47
Found twins H3 and B3 with values 47
Found twins B3 and H3 with values 47
Found twins D5 and E5 with values 36
Found twins E5 and E8 with values 36
Found twins E5 and D5 with values 36
Found twins E8 and E5 with values 36
Found twins H3 and B3 with values 47
Final puzzle state...
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In [100]:
puzzle2 = '4.....8.5.3..........7.....'\
          '.2.....6.....8.4......1....'\
          '...6.3.7.5..2.....1.4......'

solve_puzzle(puzzle2)

Initial puzzle state...
4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 

Found twins G1 and G3 with values 29
Peer I2 reduced from 679 to 67
Peer H2 reduced from 679 to 67
Peer G7 reduced from 1259 to 15
Peer G9 reduced from 1249 to 14
Peer G5 reduced from 459 to 45
Found twins G3 and G1 with values 29
Found twins H2 and I2 with values 67
Peer C2 reduced from 1569 to 159
Peer E2 reduced from 15679 to 159
Peer A2 reduced from 1679 to 19
Found twins I2 and H2 with values 67
Found twins C2 and E2 with values 59
Found twins E2 and C2 with values 59
Found twins G1 and G3 with values 29
Found twins G3 and G1 with values 29
Found twins H2 and I2 with values 67
Found twins I2 and H2 with values 67
Found twins C2 and E2 with values 59
Found twins E2 and C2 with values 59
Found twins G1 and G3 with values 29
Found t

In [ ]:
hard1 = '.....6....59.....82....8...'\
        '.45........3........6..3.54'\
        '...325..6..................'
#solve_puzzle(hard1)

In [ ]:
impossible1 = '.....5.8....6.1.43.........'\
              '.1.5........1.6...3.......5'\
              '53.....61........4.........'

impossible2 = '7....5.8....6.1.43.........'\
              '.1.5........1.6...3.......5'\
              '53.....61........4....8....'

solve_puzzle(impossible2)